In [1]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.decomposition import FastICA
from sklearn import random_projection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import homogeneity_score
from collections import Counter
import pandas as pd
import numpy as np
import random
from pytictoc import TicToc
t = TicToc()

In [2]:
vocabulary = pd.read_csv("vocab.nytimes.csv",header=None,names=['ID','WORD'] )
vocabulary.shape

(102660, 2)

In [3]:
for ite in range(44442,vocabulary.shape[0]):
    aux_word = vocabulary["WORD"][ite]
    vocabulary["WORD"][ite] = aux_word[4:]

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_3024\417027552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocabulary["WORD"][ite] = aux_word[4:]


In [4]:
#vocabulary = vocabulary[~vocabulary['WORD'].str.contains('_', na=False, regex=False)]

In [5]:
vocabulary.shape

(102660, 2)

In [6]:
corpus = pd.read_csv("docword.nytimes.txt",header=None,names=['ARTICLE','WORD','OBSERVATIONS'],sep=' ' )

In [7]:
random.seed(19830526)
corpus.shape

(69679427, 3)

In [8]:
basePoliticsWords = ['republican','democratic','senate','congress','senator','representative']
basePoliticsWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(basePoliticsWords)].values.tolist()
articlesPolitics = corpus['ARTICLE'][corpus['WORD'].isin(basePoliticsWordsIDs)].values.tolist()
print(len(articlesPolitics))
selectedPolitics = random.sample(articlesPolitics,2000)
basePoliticsWordsIDs

123418


[7886,
 10185,
 32801,
 32842,
 35020,
 35021,
 55369,
 58011,
 88569,
 88587,
 92424,
 92479]

In [9]:
baseSportWords = ['sport','player','score','competition','championship','mvp']
baseSportWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseSportWords)].values.tolist()
articlesSports = corpus['ARTICLE'][corpus['WORD'].isin(baseSportWordsIDs)].values.tolist()
print(len(articlesSports))
selectedSports = random.sample(articlesSports,2000)
baseSportWordsIDs

115066


[6101, 7518, 25453, 29176, 34560, 37194, 53285, 55276, 81367, 91809]

In [10]:
baseNYWords = ['new_york','manhattan','queens','harlem','brooklyn','new_jersey']
baseNYWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseNYWords)].values.tolist()
articlesNY = corpus['ARTICLE'][corpus['WORD'].isin(baseNYWordsIDs)].values.tolist()
print(len(articlesNY))
selectedNY = random.sample(articlesNY,2000)
baseNYWordsIDs

90905


[4723, 17861, 23359, 51396, 66202, 77181, 82350, 82418]

In [11]:
baseClimateWords = ['climate','hurricane','storm','temperature','snow','rain']
baseClimateWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseClimateWords)].values.tolist()
articlesClimate = corpus['ARTICLE'][corpus['WORD'].isin(baseClimateWordsIDs)].values.tolist()
print(len(articlesClimate))
selectedClimate = random.sample(articlesClimate,2000)
baseClimateWordsIDs

31652


[6826, 19013, 31357, 36511, 37873, 39515, 54738, 67855, 87645, 95253]

In [12]:
baseEducationWords = ['university','graduate','student','school','teacher','learning']
baseEducationWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseEducationWords)].values.tolist()
articlesEducation = corpus['ARTICLE'][corpus['WORD'].isin(baseEducationWordsIDs)].values.tolist()
print(len(articlesEducation))
selectedEducation = random.sample(articlesEducation,2000)
baseEducationWordsIDs

141960


[17033, 22130, 34498, 38097, 39341, 41880, 65170, 95365, 96227, 99200]

In [13]:
baseTechWords = ['software','internet','data','cybersecurity','technology','digital']
baseTechWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseTechWords)].values.tolist()
articlesTech = corpus['ARTICLE'][corpus['WORD'].isin(baseTechWordsIDs)].values.tolist()
print(len(articlesTech))
selectedTech = random.sample(articlesTech,2000)
baseTechWordsIDs

90350


[9422, 9602, 10811, 20570, 36639, 39398, 68584]

In [14]:

selectedArticles = selectedTech + selectedPolitics + selectedNY + selectedClimate + selectedEducation + selectedSports
selectedWords = baseTechWordsIDs + basePoliticsWordsIDs + baseNYWordsIDs + baseClimateWordsIDs + baseEducationWordsIDs + baseSportWordsIDs
print(len(selectedArticles))
#selectedArticles = list(set(selectedArticles))
#print(len(selectedArticles))
eliminating=Counter(selectedArticles)
selectedArticlesSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedArticlesSplit.append(ite)
print(len(selectedArticlesSplit))
corpus = corpus[corpus['ARTICLE'].isin(selectedArticles)]
#words_to_keep = vocabulary['ID'].tolist()
#corpus = corpus[corpus['WORD'].isin(words_to_keep)]
corpus.shape
              

12000
11413


(3216090, 3)

In [15]:
corpus = corpus.pivot(index='ARTICLE',columns='WORD',values='OBSERVATIONS')
corpus = corpus.fillna(0)
selectedWords = selectedWords + random.sample(corpus.columns.values.tolist(),10000)    
eliminating=Counter(selectedWords)
selectedWordsSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedWordsSplit.append(ite)

In [16]:
print(corpus.shape)
#corpus = corpus.loc[:, (corpus != 0).any(axis=0)]
print(len(selectedWordsSplit))
corpus = corpus[selectedWordsSplit]
print(corpus.shape)

(11702, 80879)
10039
(11702, 10039)


In [17]:

label_real = []
for row in corpus.index:
    if row in selectedPolitics:
        label_real.append(0)
    elif row in selectedTech:
        label_real.append(1)
    elif row in selectedEducation:
        label_real.append(2)
    elif row in selectedNY:
        label_real.append(3)
    elif row in selectedClimate:
        label_real.append(4)
    elif row in selectedSports:
        label_real.append(5)

In [18]:
homogeneity_sol = pd.DataFrame([['Test',0,0,0.000,0.000,0.000]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,WCSS,Time
0,Test,0,0,0.0,0.0,0.0


In [19]:
corpus = corpus.fillna(0)
corpus[corpus > 1] = 1
#columnas = list(corpus.columns)
#columnas_words = vocabulary["ID"]=

    

In [20]:
t.tic()
pca = PCA(n_components=100, random_state = 19830526)
pca.fit(corpus)
t.toc('PCA Calibration with 200 components')

PCA Calibration with 200 components 11.650379 seconds.


In [21]:
np.cumsum(pca.explained_variance_ratio_)

array([0.02092258, 0.03647718, 0.04927263, 0.05800725, 0.06648504,
       0.07362777, 0.08049479, 0.08677015, 0.09289918, 0.0988467 ,
       0.10441486, 0.10981384, 0.11517325, 0.12044074, 0.12560554,
       0.13067592, 0.13555163, 0.14026628, 0.14496009, 0.14949293,
       0.1539507 , 0.15830466, 0.16252634, 0.16672376, 0.17076541,
       0.17479284, 0.17872025, 0.18263653, 0.18642002, 0.19009205,
       0.19375883, 0.19733089, 0.20087192, 0.20431493, 0.2077308 ,
       0.21107794, 0.21439909, 0.21768218, 0.22093654, 0.22413086,
       0.22730047, 0.23043201, 0.23353029, 0.23659592, 0.23962298,
       0.24263722, 0.24562125, 0.24856243, 0.25148809, 0.25440066,
       0.25727584, 0.26012923, 0.26295331, 0.26576077, 0.26855491,
       0.2713003 , 0.27402115, 0.27670981, 0.27937382, 0.28201933,
       0.28461518, 0.2872082 , 0.28978526, 0.29235098, 0.29489191,
       0.29742531, 0.29990251, 0.30234589, 0.30476601, 0.30717449,
       0.30957227, 0.31193908, 0.31428174, 0.31660059, 0.31887

In [22]:
sum(pca.explained_variance_ratio_)

0.3702156481176338

In [23]:
t.tic()
corpus_pca = pca.transform(corpus)
t.toc('PCA transform took')

PCA transform took 1.458254 seconds.


In [24]:
labels_pca = []
for ite in range(1,20):
    for iteCom in range(1,50):
        t.tic()
        kmeans_pca = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_pca.fit(corpus_pca[:,0:iteCom])
        labels_pca.append(kmeans_pca.labels_)
        taux = t.tocvalue()
        print("PCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["PCA",iteCom,ite,homogeneity_score(label_real, kmeans_pca.labels_),kmeans_pca.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

PCA with 1 features and 1 clusters took to run 0.4748905999999806
PCA with 2 features and 1 clusters took to run 0.017282199999982595
PCA with 3 features and 1 clusters took to run 0.017647499999981164
PCA with 4 features and 1 clusters took to run 0.0214004999999986
PCA with 5 features and 1 clusters took to run 0.01742639999997664
PCA with 6 features and 1 clusters took to run 0.017877999999996064
PCA with 7 features and 1 clusters took to run 0.02169000000000665
PCA with 8 features and 1 clusters took to run 0.02388920000001349
PCA with 9 features and 1 clusters took to run 0.034200200000015
PCA with 10 features and 1 clusters took to run 0.025807200000002695
PCA with 11 features and 1 clusters took to run 0.024577899999997044
PCA with 12 features and 1 clusters took to run 0.019663900000011836
PCA with 13 features and 1 clusters took to run 0.021724399999982325
PCA with 14 features and 1 clusters took to run 0.019882899999998926
PCA with 15 features and 1 clusters took to run 0.019

PCA with 25 features and 3 clusters took to run 0.24596199999999158
PCA with 26 features and 3 clusters took to run 0.1983248999999887
PCA with 27 features and 3 clusters took to run 0.2696074000000124
PCA with 28 features and 3 clusters took to run 0.21607610000000932
PCA with 29 features and 3 clusters took to run 0.2776317000000006
PCA with 30 features and 3 clusters took to run 0.238924499999996
PCA with 31 features and 3 clusters took to run 0.2522632999999814
PCA with 32 features and 3 clusters took to run 0.2547983000000045
PCA with 33 features and 3 clusters took to run 0.20566920000001687
PCA with 34 features and 3 clusters took to run 0.2607151999999928
PCA with 35 features and 3 clusters took to run 0.21017259999999283
PCA with 36 features and 3 clusters took to run 0.2868832999999995
PCA with 37 features and 3 clusters took to run 0.2908261999999979
PCA with 38 features and 3 clusters took to run 0.3132844000000148
PCA with 39 features and 3 clusters took to run 0.295047100

PCA with 49 features and 5 clusters took to run 0.46490140000000224
PCA with 1 features and 6 clusters took to run 0.1238335000000177
PCA with 2 features and 6 clusters took to run 0.17984280000001718
PCA with 3 features and 6 clusters took to run 0.17582780000000753
PCA with 4 features and 6 clusters took to run 0.16209159999999656
PCA with 5 features and 6 clusters took to run 0.13910230000001889
PCA with 6 features and 6 clusters took to run 0.2313220000000058
PCA with 7 features and 6 clusters took to run 0.19584150000000022
PCA with 8 features and 6 clusters took to run 0.22978390000000104
PCA with 9 features and 6 clusters took to run 0.21972059999998805
PCA with 10 features and 6 clusters took to run 0.23224360000000388
PCA with 11 features and 6 clusters took to run 0.23360969999998815
PCA with 12 features and 6 clusters took to run 0.2489463999999657
PCA with 13 features and 6 clusters took to run 0.2801511999999775
PCA with 14 features and 6 clusters took to run 0.22311750000

PCA with 24 features and 8 clusters took to run 0.3640705000000253
PCA with 25 features and 8 clusters took to run 0.3789025000000379
PCA with 26 features and 8 clusters took to run 0.41026840000000675
PCA with 27 features and 8 clusters took to run 0.4678253999999811
PCA with 28 features and 8 clusters took to run 0.4748414000000025
PCA with 29 features and 8 clusters took to run 0.4148155999999972
PCA with 30 features and 8 clusters took to run 0.45900940000001356
PCA with 31 features and 8 clusters took to run 0.4087165999999911
PCA with 32 features and 8 clusters took to run 0.49873439999998936
PCA with 33 features and 8 clusters took to run 0.42740379999997913
PCA with 34 features and 8 clusters took to run 0.4035220999999751
PCA with 35 features and 8 clusters took to run 0.49413789999999835
PCA with 36 features and 8 clusters took to run 0.5777702000000318
PCA with 37 features and 8 clusters took to run 0.5217130999999995
PCA with 38 features and 8 clusters took to run 0.5092384

PCA with 48 features and 10 clusters took to run 0.6937593999999763
PCA with 49 features and 10 clusters took to run 0.6215793000000076
PCA with 1 features and 11 clusters took to run 0.14825130000002673
PCA with 2 features and 11 clusters took to run 0.25812889999997424
PCA with 3 features and 11 clusters took to run 0.37810000000001764
PCA with 4 features and 11 clusters took to run 0.33019110000003593
PCA with 5 features and 11 clusters took to run 0.31798619999995026
PCA with 6 features and 11 clusters took to run 0.3625496999999882
PCA with 7 features and 11 clusters took to run 0.3810199000000125
PCA with 8 features and 11 clusters took to run 0.364078099999972
PCA with 9 features and 11 clusters took to run 0.3426523999999631
PCA with 10 features and 11 clusters took to run 0.39455420000001595
PCA with 11 features and 11 clusters took to run 0.4964548000000377
PCA with 12 features and 11 clusters took to run 0.5278374000000099
PCA with 13 features and 11 clusters took to run 0.4

PCA with 22 features and 13 clusters took to run 0.4788381000000186
PCA with 23 features and 13 clusters took to run 0.5796799000000306
PCA with 24 features and 13 clusters took to run 0.5297107000000096
PCA with 25 features and 13 clusters took to run 0.6516285999999809
PCA with 26 features and 13 clusters took to run 0.7589195999999561
PCA with 27 features and 13 clusters took to run 0.5711056999999755
PCA with 28 features and 13 clusters took to run 0.7015025000000037
PCA with 29 features and 13 clusters took to run 0.7662496000000374
PCA with 30 features and 13 clusters took to run 0.868541700000037
PCA with 31 features and 13 clusters took to run 0.6873390000000086
PCA with 32 features and 13 clusters took to run 0.8953231000000414
PCA with 33 features and 13 clusters took to run 0.7487749000000008
PCA with 34 features and 13 clusters took to run 0.6095111999999858
PCA with 35 features and 13 clusters took to run 0.6345248000000083
PCA with 36 features and 13 clusters took to run 

PCA with 45 features and 15 clusters took to run 0.9427413000000229
PCA with 46 features and 15 clusters took to run 0.7990174000000252
PCA with 47 features and 15 clusters took to run 1.0581579999999917
PCA with 48 features and 15 clusters took to run 0.9085961000000111
PCA with 49 features and 15 clusters took to run 1.058478100000002
PCA with 1 features and 16 clusters took to run 0.16280029999995804
PCA with 2 features and 16 clusters took to run 0.39754149999998845
PCA with 3 features and 16 clusters took to run 0.3772744999999986
PCA with 4 features and 16 clusters took to run 0.36051080000004276
PCA with 5 features and 16 clusters took to run 0.37437710000006064
PCA with 6 features and 16 clusters took to run 0.4365376999999171
PCA with 7 features and 16 clusters took to run 0.5413327999999638
PCA with 8 features and 16 clusters took to run 0.46678960000008374
PCA with 9 features and 16 clusters took to run 0.41154770000002827
PCA with 10 features and 16 clusters took to run 0.4

PCA with 19 features and 18 clusters took to run 0.7968916999999465
PCA with 20 features and 18 clusters took to run 1.006235199999992
PCA with 21 features and 18 clusters took to run 1.0730596999999307
PCA with 22 features and 18 clusters took to run 0.9061703000000989
PCA with 23 features and 18 clusters took to run 0.6683179999999993
PCA with 24 features and 18 clusters took to run 0.7585477999999739
PCA with 25 features and 18 clusters took to run 0.6773484999999937
PCA with 26 features and 18 clusters took to run 0.6783431999999721
PCA with 27 features and 18 clusters took to run 0.7232204999999112
PCA with 28 features and 18 clusters took to run 0.7845802000000504
PCA with 29 features and 18 clusters took to run 0.7804110999999239
PCA with 30 features and 18 clusters took to run 0.8088510000000042
PCA with 31 features and 18 clusters took to run 0.7070770000000266
PCA with 32 features and 18 clusters took to run 0.8143107000000782
PCA with 33 features and 18 clusters took to run 

In [25]:
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,WCSS,Time
0,Test,0,0,0.000000,0.000000,0.000000
0,PCA,1,1,0.000000,8080.474299,0.474891
0,PCA,2,1,0.000000,14087.786143,0.017282
0,PCA,3,1,0.000000,19029.496353,0.017647
0,PCA,4,1,0.000000,22402.879333,0.021400
...,...,...,...,...,...,...
0,PCA,45,19,0.219420,69943.550905,1.002039
0,PCA,46,19,0.226902,71205.423048,1.166764
0,PCA,47,19,0.227546,72264.978876,1.049988
0,PCA,48,19,0.208645,73422.729313,1.185863


In [26]:
labels = []
for ite in range(1,20):
    t.tic()
    kmeans = KMeans(n_clusters = ite, random_state = 19830526)
    kmeans.fit(corpus)
    labels.append(kmeans.labels_)
    taux = t.tocvalue()
    print("RAW with "+str(ite)+" clusters took to run " + str(taux))
    new_row = pd.DataFrame([["RAW",0,ite,homogeneity_score(label_real, kmeans.labels_),kmeans.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
    homogeneity_sol = pd.concat([homogeneity_sol,new_row])

RAW with 1 clusters took to run 6.795985999999971
RAW with 2 clusters took to run 30.448985300000004
RAW with 3 clusters took to run 35.832271799999944
RAW with 4 clusters took to run 39.701401499999974
RAW with 5 clusters took to run 52.79266889999997
RAW with 6 clusters took to run 46.777462499999956
RAW with 7 clusters took to run 55.52884400000005
RAW with 8 clusters took to run 59.07387630000005
RAW with 9 clusters took to run 62.7884411
RAW with 10 clusters took to run 85.58171470000002
RAW with 11 clusters took to run 74.65363479999996
RAW with 12 clusters took to run 71.76610800000003
RAW with 13 clusters took to run 84.3295376000001
RAW with 14 clusters took to run 86.02722610000001
RAW with 15 clusters took to run 97.99327519999997
RAW with 16 clusters took to run 84.45913590000009
RAW with 17 clusters took to run 81.94988719999992
RAW with 18 clusters took to run 97.03406769999992
RAW with 19 clusters took to run 89.91222700000003


In [27]:
#ICA = FastICA(n_components=200,max_iter=1000)
#corpus_ica=ICA.fit_transform(corpus)

In [28]:
labels_ica = []
for iteCom in range(1,50):
    t.tic()
    ICA = FastICA(n_components=iteCom,max_iter=1000, random_state = 19830526)
    corpus_ica=ICA.fit_transform(corpus)        
    t.toc("ICA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        kmeans_ica = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_ica.fit(corpus_ica)
        labels_ica.append(kmeans_ica.labels_)
        taux = t.tocvalue()
        print("ICA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["ICA",iteCom,ite,homogeneity_score(label_real, kmeans_ica.labels_),kmeans_ica.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 1 components took  747.791145 seconds.
ICA with 1 features and 1 clusters took to run 0.023065500000029715
ICA with 1 features and 2 clusters took to run 0.03877699999975448
ICA with 1 features and 3 clusters took to run 0.06155570000009902
ICA with 1 features and 4 clusters took to run 0.10284240000009959
ICA with 1 features and 5 clusters took to run 0.10237660000029791
ICA with 1 features and 6 clusters took to run 0.09489750000011554
ICA with 1 features and 7 clusters took to run 0.15161609999995562
ICA with 1 features and 8 clusters took to run 0.10838299999977608
ICA with 1 features and 9 clusters took to run 0.1395220999997946
ICA with 1 features and 10 clusters took to run 0.12897680000014589
ICA with 1 features and 11 clusters took to run 0.19641109999975015
ICA with 1 features and 12 clusters took to run 0.1742023000001609
ICA with 1 features and 13 clusters took to run 0.15139750000025742
ICA with 1 features and 14 clusters took to run 0.15005459999974846

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 2 components took  765.965374 seconds.
ICA with 2 features and 1 clusters took to run 0.9710598000001482
ICA with 2 features and 2 clusters took to run 3.417263400000138
ICA with 2 features and 3 clusters took to run 2.7162201000001005
ICA with 2 features and 4 clusters took to run 4.28851080000004
ICA with 2 features and 5 clusters took to run 1.7711050000002615
ICA with 2 features and 6 clusters took to run 0.15427349999981743
ICA with 2 features and 7 clusters took to run 0.1511395000002267
ICA with 2 features and 8 clusters took to run 0.21299339999995937
ICA with 2 features and 9 clusters took to run 0.15151790000027177
ICA with 2 features and 10 clusters took to run 0.18012709999993604
ICA with 2 features and 11 clusters took to run 0.22175370000013572
ICA with 2 features and 12 clusters took to run 0.2574786000000131
ICA with 2 features and 13 clusters took to run 9.52231380000012
ICA with 2 features and 14 clusters took to run 3.5059487999997145
ICA with 2 f

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


...

ICA calibration for 47 components took  806.810142 seconds.
ICA with 47 features and 1 clusters took to run 0.03586999999242835
ICA with 47 features and 2 clusters took to run 0.13703099999111146
ICA with 47 features and 3 clusters took to run 0.1829105999931926
ICA with 47 features and 4 clusters took to run 0.18094390000624117
ICA with 47 features and 5 clusters took to run 0.210365399994771
ICA with 47 features and 6 clusters took to run 0.29259079998882953
ICA with 47 features and 7 clusters took to run 0.3048465000028955
ICA with 47 features and 8 clusters took to run 0.33553030001348816
ICA with 47 features and 9 clusters took to run 0.40550800001074094
ICA with 47 features and 10 clusters took to run 0.4867707999947015
ICA with 47 features and 11 clusters took to run 0.5213387999974657
ICA with 47 features and 12 clusters took to run 0.38669990000198595
ICA with 47 features and 13 clusters took to run 0.3874885999975959
ICA with 47 features and 14 clusters took to run 1.04293550

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 48 components took  525.653531 seconds.
ICA with 48 features and 1 clusters took to run 0.0465179999882821
ICA with 48 features and 2 clusters took to run 0.17120490000525024
ICA with 48 features and 3 clusters took to run 0.16178700000455137
ICA with 48 features and 4 clusters took to run 0.1928355999989435
ICA with 48 features and 5 clusters took to run 0.20620129999588244
ICA with 48 features and 6 clusters took to run 0.22455160001118202
ICA with 48 features and 7 clusters took to run 0.27449629998591263
ICA with 48 features and 8 clusters took to run 0.28360840000095777
ICA with 48 features and 9 clusters took to run 0.3619992000021739
ICA with 48 features and 10 clusters took to run 0.4064332999987528
ICA with 48 features and 11 clusters took to run 0.36392079999495763
ICA with 48 features and 12 clusters took to run 0.4024083999975119
ICA with 48 features and 13 clusters took to run 0.43459319999965373
ICA with 48 features and 14 clusters took to run 0.485658

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 49 components took  827.012767 seconds.
ICA with 49 features and 1 clusters took to run 0.1014292000036221
ICA with 49 features and 2 clusters took to run 0.45700020001095254
ICA with 49 features and 3 clusters took to run 0.4152014999999665
ICA with 49 features and 4 clusters took to run 0.5437302000063937
ICA with 49 features and 5 clusters took to run 0.5909438000089722
ICA with 49 features and 6 clusters took to run 0.5688919999956852
ICA with 49 features and 7 clusters took to run 0.7033492000045953
ICA with 49 features and 8 clusters took to run 0.9114741000084905
ICA with 49 features and 9 clusters took to run 0.8318293999909656
ICA with 49 features and 10 clusters took to run 1.1033416999998735
ICA with 49 features and 11 clusters took to run 0.9952013000001898
ICA with 49 features and 12 clusters took to run 1.0555008000083035
ICA with 49 features and 13 clusters took to run 1.1993350999982795
ICA with 49 features and 14 clusters took to run 1.2799398999923

In [29]:
print(corpus.shape)
print(len(label_real))

(11702, 10039)
11702


In [30]:
RCA = random_projection.SparseRandomProjection()
corpus_rca = RCA.fit_transform(corpus)

In [31]:
labels_rca = []
for iteCom in range(1,50):
    t.tic()
    RCA = random_projection.SparseRandomProjection(n_components=iteCom, random_state = 19830526)
    corpus_rca = RCA.fit_transform(corpus)
    t.toc("RCA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        kmeans_rca = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_rca.fit(corpus_rca)
        labels_rca.append(kmeans_rca.labels_)
        taux = t.tocvalue()
        print("RCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["RCA",iteCom,ite,homogeneity_score(label_real, kmeans_rca.labels_),kmeans_rca.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

RCA calibration for 1 components took  0.952836 seconds.
RCA with 1 features and 1 clusters took to run 0.038853599995491095
RCA with 1 features and 2 clusters took to run 0.07766510000510607
RCA with 1 features and 3 clusters took to run 0.047551499999826774
RCA with 1 features and 4 clusters took to run 0.09729360000346787
RCA with 1 features and 5 clusters took to run 0.05879310000455007
RCA with 1 features and 6 clusters took to run 0.10864720000245143
RCA with 1 features and 7 clusters took to run 0.07730790000641719
RCA with 1 features and 8 clusters took to run 0.11178900000231806


C:\Users\Alvaro\AppData\Local\Temp\ipykernel_3024\1857541420.py:10: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  kmeans_rca.fit(corpus_rca)
C:\Users\Alvaro\AppData\Local\Temp\ipykernel_3024\1857541420.py:10: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans_rca.fit(corpus_rca)


RCA with 1 features and 9 clusters took to run 0.1455294000043068
RCA with 1 features and 10 clusters took to run 0.14228729999740608


C:\Users\Alvaro\AppData\Local\Temp\ipykernel_3024\1857541420.py:10: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (11). Possibly due to duplicate points in X.
  kmeans_rca.fit(corpus_rca)
C:\Users\Alvaro\AppData\Local\Temp\ipykernel_3024\1857541420.py:10: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (12). Possibly due to duplicate points in X.
  kmeans_rca.fit(corpus_rca)


RCA with 1 features and 11 clusters took to run 0.1596317000075942
RCA with 1 features and 12 clusters took to run 0.15608590000192635


...

RCA with 20 features and 7 clusters took to run 0.25274300000455696
RCA with 20 features and 8 clusters took to run 0.27427439999883063
RCA with 20 features and 9 clusters took to run 0.27847920000203885
RCA with 20 features and 10 clusters took to run 0.29448969999793917
RCA with 20 features and 11 clusters took to run 0.34397449999232776
RCA with 20 features and 12 clusters took to run 0.3369428000005428
RCA with 20 features and 13 clusters took to run 0.3576191000029212
RCA with 20 features and 14 clusters took to run 0.3933770000003278
RCA with 20 features and 15 clusters took to run 0.4525895999977365
RCA with 20 features and 16 clusters took to run 0.47030200000153854
RCA with 20 features and 17 clusters took to run 0.47494249998999294
RCA with 20 features and 18 clusters took to run 0.49966780000249855
RCA with 20 features and 19 clusters took to run 0.5524504000059096
RCA calibration for 21 components took  0.625701 seconds.
RCA with 21 features and 1 clusters took to run 0.024

RCA with 26 features and 9 clusters took to run 3.775355899997521
RCA with 26 features and 10 clusters took to run 3.7557717000017874
RCA with 26 features and 11 clusters took to run 3.768662900009076
RCA with 26 features and 12 clusters took to run 4.308230199996615
RCA with 26 features and 13 clusters took to run 3.5208718000067165
RCA with 26 features and 14 clusters took to run 4.047611200003303
RCA with 26 features and 15 clusters took to run 4.752724999998463
RCA with 26 features and 16 clusters took to run 4.532202499991399
RCA with 26 features and 17 clusters took to run 4.864937099991948
RCA with 26 features and 18 clusters took to run 5.033679400003166
RCA with 26 features and 19 clusters took to run 5.384170300007099
RCA calibration for 27 components took  1.767037 seconds.
RCA with 27 features and 1 clusters took to run 0.16915760000119917
RCA with 27 features and 2 clusters took to run 0.6242740000016056
RCA with 27 features and 3 clusters took to run 1.5043972999992548
RC

RCA with 32 features and 12 clusters took to run 0.9444417000049725
RCA with 32 features and 13 clusters took to run 1.078277300010086
RCA with 32 features and 14 clusters took to run 1.3330838999972912
RCA with 32 features and 15 clusters took to run 1.3591842000023462
RCA with 32 features and 16 clusters took to run 1.6126541999983601
RCA with 32 features and 17 clusters took to run 1.5279950999974972
RCA with 32 features and 18 clusters took to run 1.6301445999997668
RCA with 32 features and 19 clusters took to run 1.6472211999935098
RCA calibration for 33 components took  0.990121 seconds.
RCA with 33 features and 1 clusters took to run 0.04701759999443311
RCA with 33 features and 2 clusters took to run 0.3647643000003882
RCA with 33 features and 3 clusters took to run 0.7374003999866545
RCA with 33 features and 4 clusters took to run 0.7628914999950211
RCA with 33 features and 5 clusters took to run 1.0252153000037652
RCA with 33 features and 6 clusters took to run 0.7063756999996

RCA with 38 features and 15 clusters took to run 0.6797465000126977
RCA with 38 features and 16 clusters took to run 0.7016026999917813
RCA with 38 features and 17 clusters took to run 0.6226093000004767
RCA with 38 features and 18 clusters took to run 0.7867298999917693
RCA with 38 features and 19 clusters took to run 0.7309412000031443
RCA calibration for 39 components took  0.647634 seconds.
RCA with 39 features and 1 clusters took to run 0.03520149999530986
RCA with 39 features and 2 clusters took to run 0.18501510001078714
RCA with 39 features and 3 clusters took to run 0.23163759999442846
RCA with 39 features and 4 clusters took to run 0.24813100000028498
RCA with 39 features and 5 clusters took to run 0.2871039000019664
RCA with 39 features and 6 clusters took to run 0.3923464999970747
RCA with 39 features and 7 clusters took to run 0.39904489999753423
RCA with 39 features and 8 clusters took to run 0.41509450000012293
RCA with 39 features and 9 clusters took to run 0.5054066000

RCA with 44 features and 17 clusters took to run 0.677343600007589
RCA with 44 features and 18 clusters took to run 0.7405143000069074
RCA with 44 features and 19 clusters took to run 0.7830950999923516
RCA calibration for 45 components took  0.673661 seconds.
RCA with 45 features and 1 clusters took to run 0.03018460000748746
RCA with 45 features and 2 clusters took to run 0.16620469999907073
RCA with 45 features and 3 clusters took to run 0.25376419999520294
RCA with 45 features and 4 clusters took to run 0.2964248000062071
RCA with 45 features and 5 clusters took to run 0.27328719999059103
RCA with 45 features and 6 clusters took to run 0.4482808000029763
RCA with 45 features and 7 clusters took to run 0.39155810000374913
RCA with 45 features and 8 clusters took to run 0.40785329999926034
RCA with 45 features and 9 clusters took to run 0.43443480000132695
RCA with 45 features and 10 clusters took to run 0.5747207999957027
RCA with 45 features and 11 clusters took to run 0.6504608999

In [32]:
labels_lda = []
   
for iteCom in range(1,6):
    t.tic()
    LDA = LinearDiscriminantAnalysis(n_components=iteCom)
    corpus_lda = LDA.fit_transform(corpus, label_real)
    t.toc("LDA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        kmeans_lda = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_lda.fit(corpus_lda)
        labels_lda.append(kmeans_lda.labels_)    
        taux = t.tocvalue()
        print("LCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["LDA",iteCom,ite,homogeneity_score(label_real, kmeans_lda.labels_),kmeans_lda.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

LDA calibration for 1 components took  674.438574 seconds.
LCA with 1 features and 1 clusters took to run 0.01414699999440927
LCA with 1 features and 2 clusters took to run 0.02415139999357052
LCA with 1 features and 3 clusters took to run 0.04239159999997355
LCA with 1 features and 4 clusters took to run 0.054827499989187345
LCA with 1 features and 5 clusters took to run 0.0826501000119606
LCA with 1 features and 6 clusters took to run 0.06539259999408387
LCA with 1 features and 7 clusters took to run 0.07805340000777505
LCA with 1 features and 8 clusters took to run 0.0903192000114359
LCA with 1 features and 9 clusters took to run 0.09116900000663009
LCA with 1 features and 10 clusters took to run 0.10261950000131037
LCA with 1 features and 11 clusters took to run 0.11042880000604782
LCA with 1 features and 12 clusters took to run 0.11939490000077058
LCA with 1 features and 13 clusters took to run 0.12739300000248477
LCA with 1 features and 14 clusters took to run 0.14031260000774637

In [33]:
df = pd.DataFrame(label_real, index=corpus.index ,columns =['Real'])

In [34]:
homogeneity_sol.to_csv("BagOfWordsClusteringResultsKMeans.csv")

In [43]:
df_ica = pd.DataFrame(labels_ica, columns=corpus.index).T
df_pca = pd.DataFrame(labels_pca, columns=corpus.index).T
df_rca = pd.DataFrame(labels_rca, columns=corpus.index).T
df_lda = pd.DataFrame(labels_lda, columns=corpus.index).T
df_raw = pd.DataFrame(labels, columns=corpus.index).T 
df.to_csv("actuals.csv")
df_ica.to_csv("ica.csv")
df_pca.to_csv("pca.csv")
df_rca.to_csv("rca.csv")
df_lda.to_csv("lda.csv")
df_raw.to_csv("raw.csv")

In [44]:
print(homogeneity_sol)

   Algorithm  Components  Clusters  Homogeneity          WCSS      Time
0       Test           0         0     0.000000      0.000000  0.000000
0        PCA           1         1     0.000000   8080.474299  0.474891
0        PCA           2         1     0.000000  14087.786143  0.017282
0        PCA           3         1     0.000000  19029.496353  0.017647
0        PCA           4         1     0.000000  22402.879333  0.021400
..       ...         ...       ...          ...           ...       ...
0        LDA           5        15     0.953708  38780.959624  0.212825
0        LDA           5        16     0.954129  38013.638129  0.224829
0        LDA           5        17     0.953457  37421.205365  0.227846
0        LDA           5        18     0.954627  36727.276052  0.233755
0        LDA           5        19     0.954853  36092.118688  0.271475

[2908 rows x 6 columns]
